In [61]:
import re
import os
import time
import json
import pandas as pd
import requests
import socket
import datetime
import datetime as dt
import warnings
from datetime import datetime
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup
import urllib.request
import newspaper
from newspaper import Article, fulltext
import matplotlib.pyplot as plt
import plotly.plotly as py
import cufflinks as cf
import plotly.tools as tls
from pytrends.request import TrendReq
import pytz
import feedparser


from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly
import plotly.graph_objs as go

import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#plotly.offline.init_notebook_mode()
#init_notebook_mode(connected=True)         # initiate notebook for offline plot

#google_username = ""
#google_password = ""
#pytrends = TrendReq()
#cf.go_offline()

headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"}
#warnings.filterwarnings("ignore")

#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:90% !important; }</style>"))

## Done: Live Coin Watch, Coindesk, Forexlive, Coinlore, Cointelegraph, 

# Live Coin Watch


In [2]:
def getnews(url):

    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"}
    url ='https://news.livecoinwatch.com/'
    res = requests.get(url,headers=headers)

    soup = bs(res.content,'lxml')
    df = pd.DataFrame()
    x=soup.findAll('div',attrs={'class':'td_block_inner td-column-2'})

    Title =[]
    Link =[]
    Text =[]
    Times =[]
    Comment =[]
    for something in x[0].find_all("a"):
        title = something.get('title')
        link = something.get('href')
        Link.append(link)
        Title.append(title)
        foo = something.find_parent('div')
        i = foo.find(class_='td-excerpt')
        if i is not None:
            Text.append(i.text.replace('\r\n',''))
        else:
            Text.append('No Text')   
        b = something.find_parent('div')
        c = b.find('time')
        if c is not None:
            d = c.get('datetime')
            Times.append(pd.to_datetime(d))
        else:
            Times.append('No Time')

        k = something.find_parent('div').find('div',class_='td-module-comments')
        if k is not None:
            Comment.append(k.text)
        else:
            Comment.append('No comment')



    df['Title'] = Title
    df['Time'] = Times
    df['Links'] = Link  
    df['Text'] = Text

    df['Comment'] = Comment

    df = df.dropna().drop_duplicates(subset=['Title'],keep='last')
    return df

In [3]:
url = 'https://news.livecoinwatch.com/'
LiveCoinWatch = getnews(url)
#LiveCoinWatch.reset_index(inplace = True)
LiveCoinWatch

,Title,Time,Links,Text,Comment
1,"This Week In Crypto: BTC Tops $10,000, Ripple ...",2019-06-22 05:11:57+00:00,https://news.livecoinwatch.com/week-crypto-btc...,No Text,0
6,New cryptocurrencies on 1xBit: stablecoins com...,2019-06-17 23:00:14+00:00,https://news.livecoinwatch.com/new-cryptocurre...,One of the leading cryptoc...,0
11,This Week In Crypto: Google Lauds Ethereum-bas...,2019-06-15 18:00:50+00:00,https://news.livecoinwatch.com/this-week-in-cr...,This Week In Crypto is a w...,0
16,"This Week In Crypto: India Bitcoin Ban, EOS Vo...",2019-06-08 04:38:00+00:00,https://news.livecoinwatch.com/this-week-crypt...,This Week In Crypto is a w...,0
21,"This Week In Crypto: Bitcoin Bulls Roar, Priva...",2019-06-01 01:50:08+00:00,https://news.livecoinwatch.com/week-crypto-bit...,This Week In Crypto is a w...,0


In [59]:
LiveCoinWatch['Links'][1]

'https://news.livecoinwatch.com/week-crypto-btc-10000-ripple-moneygram-ethereum-2-0/'

In [62]:
url ='https://news.livecoinwatch.com/week-crypto-btc-10000-ripple-moneygram-ethereum-2-0/'
res = requests.get(url,headers=headers)

# Scrape Coindesk

In [4]:
def getnews(url):
    
    res = requests.get(url)
    soup = bs(res.content,'lxml')
    df = pd.DataFrame()
    x=soup.findAll('div',id='body-container')
    b=x[0].find_all('a')
    Title =[]
    for i in b:
        c = i.get('title')
        Title.append(c)
    
    link =[]
    for i in b:
        c = i.get('href')
        link.append(c)
        
    Times =[]
    for i in b:
        time = i.find('time')
        time=str(time)
        time = time.split("\"")[1].split('Z')[0]
        time = pd.to_datetime(time)
        time = time.tz_convert('Europe/Berlin')
        Times.append(time)
        
    Text =[]
    for i in b:
        if i.p is None:
            Text.append('No Text')
        else:
            Text.append(i.p.text)
    
    
    df['Title'] = Title
    df['Text'] = Text
    df['Time']=Times
    df['Links'] = link
    
    return df

In [5]:
url = 'https://www.coindesk.com/'
Coindesk=getnews(url)
#Coindesk.set_index('Time',inplace = True)
Coindesk

,Title,Text,Time,Links
0,Trump Administration in Talks With Crypto Star...,No Text,2019-06-27 23:31:14+02:00,https://www.coindesk.com/trump-administration-...
1,A Wannabe Netflix Raised $575 Million on Ether...,No Text,2019-06-28 10:00:19+02:00,https://www.coindesk.com/a-wannabe-netflix-rai...
2,US Lawmakers Question Terrorist Use of Faceboo...,No Text,2019-06-27 23:47:47+02:00,https://www.coindesk.com/us-lawmakers-question...
3,Down $1.7K: Bitcoin’s Price Dives Amid Crypto ...,No Text,2019-06-28 03:30:42+02:00,https://www.coindesk.com/down-1-7k-bitcoin-div...
4,Irish Crypto Exchange Bitsane May Have Disappe...,Irish cryptocurrency exchange Bitsane may have...,2019-06-28 11:00:01+02:00,https://www.coindesk.com/irish-crypto-exchange...
5,tZERO Launches Second Digital Security to Trad...,"OSTKO, the Digital Voting Series A-1 Preferred...",2019-06-28 05:00:25+02:00,https://www.coindesk.com/tzero-launches-second...
6,Ethereum Co-Founder’s Polkadot Closes Token Sa...,Polkadot has closed on a private token sale th...,2019-06-27 23:55:43+02:00,https://www.coindesk.com/ethereum-co-founders-...
7,Grin Developers Agree to Alter Technical Devel...,Developers behind the cryptocurrency Grin agre...,2019-06-27 23:00:02+02:00,https://www.coindesk.com/grin-to-suspend-part-...
8,ShapeShift Founder Says Crypto Exchange Servic...,Erik Voorhees thinks that Facebook's currency ...,2019-06-27 22:00:39+02:00,https://www.coindesk.com/shapeshift-founder-sa...
9,"Bitcoin Price Takes Another Tumble, Shedding N...",The overall crypto market has shed some $80 bi...,2019-06-27 19:51:54+02:00,https://www.coindesk.com/bitcoin-price-takes-a...


# Scrape Forexlive

In [6]:
def pullnewsforexlive(url):
    df = pd.DataFrame()
    res = requests.get(url)
    soup = bs(res.content,'lxml')
    
    x = soup.find_all('div',class_='col-xs-12 col-sm-7 col-md-8 col-lg-8') #col-xs-12 col-sm-7 col-md-8 col-lg-8
    article =x[0].find_all('article')
    
    Link=[]
    Title =[]
    Text =[]
    Time =[]
    Comment=[]
    for i in article:
        link = i.find('a').get('href')
        title = i.find('a').get('title')
        Link.append(link)
        Title.append(title.replace('View: ','').replace('View Article: ',''))
        time = i.find('time').get('datetime')
        Time.append(pd.to_datetime(time))

        text = i.find(class_='teaser-text')
        if text is not None:
            Text.append(text.text)
        else:
            k = i.find(class_='artbody artbody-limit').get_text(strip=True).replace('\xa0','')
            Text.append(k.replace("\\r","")) 
            
        h= i.find(class_='ccinner')
        Comment.append(int(h.text))
    df['Title'] = Title
    df['Text'] = Text
    df['Time'] = Time
    df['Link'] = Link
    df['Comment'] = Comment
    return df

In [7]:
url = 'https://www.forexlive.com/Cryptocurrency'
Forexlive = pullnewsforexlive(url)
Forexlive.columns=['Title','Text','Time','Links','Comment']
Forexlive

,Title,Text,Time,Links,Comment
0,Bitcoin tries to restart after the 25% plunge ...,Bitcoin falls 25% from the high..The price of ...,2019-06-27 20:13:06+00:00,//www.forexlive.com/cryptocurrency/!/bitcoin-t...,2
1,"Easy come, easy go as Bitcoin falls to $10,500",Bitcoin takes a dive\r\nAfter nearly reaching ...,2019-06-27 17:25:58+00:00,//www.forexlive.com/cryptocurrency/!/easy-come...,11
2,Bitcoin: Buyers maintain near-term control aft...,Keep calm and HODL?\r\nForexLive\r\nThe sharp ...,2019-06-27 04:44:58+00:00,//www.forexlive.com/cryptocurrency/!/bitcoin-b...,5
3,Bitcoin surges past $13000 now.,Took at $12000 earlier today and now $13000. ...,2019-06-26 16:55:12+00:00,//www.forexlive.com/cryptocurrency/!/bitcoin-s...,7
4,Bitcoin rally fueled mostly by crypto enthusia...,If crypto can recapture the public's imaginati...,2019-06-26 14:21:10+00:00,//www.forexlive.com/cryptocurrency/!/bitcoin-r...,2
5,"Bitcoin heads up through 12,000 USD",While gold has had a poor session here in Asia...,2019-06-26 03:24:58+00:00,//www.forexlive.com/cryptocurrency/!/bitcoin-h...,0
6,Bitcoin got close to 50% midpoint today,50% of the move down from the December 2017 hi...,2019-06-25 16:15:53+00:00,//www.forexlive.com/cryptocurrency/!/bitcoin-g...,0
7,Futures bets point to growing speculative shor...,Hedge funds take positions against small trade...,2019-06-25 13:32:12+00:00,//www.forexlive.com/cryptocurrency/!/futures-b...,14
8,Fed Board reject Moore to start a cryptocurren...,Stephen Moore missed out on being appointed to...,2019-06-24 23:55:41+00:00,//www.forexlive.com/cryptocurrency/!/fed-board...,1
9,Bitcoin moves up to unchanged. Correction is ...,The $11284.44 was the high today\r\nThe price ...,2019-06-24 16:23:25+00:00,//www.forexlive.com/cryptocurrency/!/bitcoin-m...,0


# Coinlore

In [8]:
def getnews(url):
    res = requests.get(url)
    soup = bs(res.content,'lxml')
    df = pd.DataFrame()
    x=soup.findAll('div',class_='news-item')
    
    General =[]
    for i in x:
        k=i.text
        General.append(k)

    Time =[]
    Title =[]
    Date =[]
    for i in General:
        time = i[1:4].replace('\n','')
        title = i[5:].replace('\n','').replace('\u200a—\u200a','-')
        Time.append(time)
        Title.append(title)
    
    df['Time'] = Time
        
    Date =[]
    for x in df['Time']:
        if 'm' in x:
            x = x.replace('m','')
            time = dt.datetime.now()- dt.timedelta(minutes=int(x))
            Date.append(time)
        elif 'h' in x:
            x = x.replace('h','')
            time = dt.datetime.now()- dt.timedelta(hours=int(x))
            Date.append(time)
    
    df['Time'] = Date
    df['Title'] = Title
    
    return df

In [9]:
url = 'https://www.coinlore.com/crypto-news'
Coinlore = getnews(url)
Coinlore

,Time,Title
0,2019-06-28 11:02:55.421684,YGGDRASH TestNet Beta Release Schedule YGGDRASH
1,2019-06-28 11:01:55.421684,MAJOR Announcement: EdenChain Mainnet is up an...
2,2019-06-28 11:00:55.421684,ZILLA Update-June 28th Zilla
3,2019-06-28 10:57:55.421684,Best ways to top up your fiat wallet in Singap...
4,2019-06-28 10:57:55.421684,Best ways to top up your fiat wallet in Singap...
5,2019-06-28 10:55:55.421684,When Will The Next Bitcoin Bull Market Come? OKB
6,2019-06-28 10:55:55.421684,When Will The Next Bitcoin Bull Market Come?
7,2019-06-28 10:53:55.421684,ICON TX CHALLENGE : ICON’s Got Talent ICON
8,2019-06-28 10:51:55.421684,"Despite The Recent Plunge, Here’s Why Bitcoin ..."
9,2019-06-28 10:49:55.421684,The last few months at OPUS-Recap Opus


# Cointelegraph

In [10]:
def getcointele():
    df = pd.DataFrame()
    d = feedparser.parse('https://cointelegraph.com/rss')

    Link =[]
    Title =[]
    Date=[]
    for i in d.entries:
        Title.append(i['title'])
        Link.append(i['link']) 
        Date.append(pd.to_datetime(i['published']))
    df['Time'] = Date
    df['Title'] = Title
    

    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"}  
    res = requests.get('https://cointelegraph.com/',headers=headers)
    soup = bs(res.content,'lxml')
    x=soup.find_all('p',class_='post-preview-item-card__text')
    Text=[]
    Titlemini = []
    for i in x:
        Text.append(i.text)
        z = i.find_parent('div').find_all('a')
        for i in z:
            if i.get('title') is not None:
                Titlemini.append(i.get('title'))

    df1 = pd.DataFrame()
    df1['Title'] = Titlemini
    df['Link'] = Link
    df1['Text'] = Text
    df2 = df1.merge(df)

    return df2

In [11]:
Cointelegraph=getcointele()
#Cointelegraph.set_index('Time',inplace = True)
Cointelegraph.columns=['Title','Text','Time','Links']
Cointelegraph

,Title,Text,Time,Links
0,Bitcoin Mining is Now More Competitive Than Ev...,Miners are dedicating more and more resources...,2019-06-28 10:08:00+01:00,https://cointelegraph.com/news/bitcoin-mining-...
1,Central Bank of Uganda Warns Decentralized Cry...,The deputy governor of the Bank of Uganda has...,2019-06-28 10:01:00+01:00,https://cointelegraph.com/news/central-bank-of...
2,"Bitcoin Price Back Above $11,000 as Dip 'Consi...","Coming off from highs of $13,800 while bounci...",2019-06-28 09:23:00+01:00,https://cointelegraph.com/news/bitcoin-price-b...
3,Exit Scam? Dublin-Based Exchange Bitsane Vanis...,Ireland-based cryptocurrency exchange Bitsane...,2019-06-28 09:10:00+01:00,https://cointelegraph.com/news/exit-scam-dubli...
4,Iran Bitcoin Miners Set Up Shop in Mosques Ami...,Mosques’ free electricity make them an attrac...,2019-06-28 08:39:00+01:00,https://cointelegraph.com/news/iran-bitcoin-mi...
5,SEC-Registered Clearing House Brings Crypto Tr...,Apex Clearing and its crypto investment subsi...,2019-06-28 08:22:00+01:00,https://cointelegraph.com/news/sec-registered-...
6,Charles Hoskinson Thinks Cardano Will Prevail ...,Cardano founder Charles Hoskinson sees emergi...,2019-06-28 02:15:00+01:00,https://cointelegraph.com/news/charles-hoskins...
7,Ethereum Classic Devs Building a ‘Chainhopping...,ETC Labs announces partnership with Metronome...,2019-06-28 01:35:00+01:00,https://cointelegraph.com/news/etc-labs-unveil...
8,House Reps Question FinCEN Director on Libra’s...,House Reps question FinCEN Director Kenneth B...,2019-06-28 00:00:00+01:00,https://cointelegraph.com/news/house-reps-ques...
9,Seychelles to Host First Regulated Tokenized S...,MERJ Exchange of the Seychelles to offer the ...,2019-06-27 22:30:00+01:00,https://cointelegraph.com/news/seychelles-to-h...


# Coinstat

In [12]:
# Function with API to pull news data from Coinstats

def pullnews_coinstats(url):
    
    response = requests.get(url)
    data = response.json()
    data1 = data["news"]
    coinstat_news = pd.DataFrame(data1)
    
    # Parse data from API
    coinstat_news["feedDate"] = pd.to_datetime(coinstat_news["feedDate"],unit="ms").dt.round("T")
    coinstat_news = coinstat_news.sort_index(ascending=False)
    coinstat_news = coinstat_news.assign(noofnews=lambda x: len(x["title"])/len(x["title"]))
    #coinstat_news["time"] = coinstat_news["feedDate"].dt.round("15min")
    coinstat_news["time_60"] = coinstat_news["feedDate"].dt.round("60min")
    coinstat_news["number"] = coinstat_news.groupby(["time_60"])["noofnews"].count()
    #coinstat_news["number"] = coinstat_news["time"].groupby(pd.TimeGrouper(freq=("30Min"))).count()
    coinstat_news["count"] = coinstat_news["noofnews"].cumsum()
    coinstat_news= coinstat_news.fillna(0)
    coinstat_news.set_index("feedDate", inplace=True)

    
    return coinstat_news


In [13]:
url = 'https://api.coinstats.app/public/v1/news?skip=0&limit=20'
coinstat_news = pullnews_coinstats(url)

#
coinstat_news_number = coinstat_news.groupby(["time_60"])["noofnews"].count()

In [14]:
Coinstat = coinstat_news[["title","link"]] 
Coinstat.reset_index(inplace = True)
Coinstat.columns=['Time','Title','Links']
Coinstat

,Time,Title,Links
0,2019-06-28 08:14:00,Mike Novogratz Sells Part of His Bitcoin Befor...,https://u.today/mike-novogratz-sells-part-of-h...
1,2019-06-28 08:19:00,tZERO Launches Cryptocurrency Wallet and Tradi...,https://btcmanager.com/tzero-cryptocurrency-wa...
2,2019-06-28 08:22:00,ICON Development Roadmap Update — Jun 2019,https://www.reddit.com/r/CryptoCurrency/commen...
3,2019-06-28 08:23:00,"Bitcoin Price Back Above $11,000 as Dip 'Consi...",https://cointelegraph.com/news/bitcoin-price-b...
4,2019-06-28 08:23:00,Bitcoin Cash is by far the cheapest widely ado...,https://www.reddit.com/r/btc/comments/c6hnoh/b...
5,2019-06-28 08:24:00,"Blockchain to Boost ATM Security, First Blockc...",https://coingape.com/blockchain-enabled-5fa-ve...
6,2019-06-28 08:27:00,This Happened On Planet Earth...June 2019...En...,https://www.reddit.com/r/CryptoCurrency/commen...
7,2019-06-28 08:30:00,"Can Bitcoin (BTC) Still Rise To $20,000?",https://cryptodaily.co.uk/2019/06/can-bitcoin-...
8,2019-06-28 08:37:00,Can blockchain address the Medicare-for-all op...,https://www.cryptopolitan.com/can-blockchain-a...
9,2019-06-28 08:40:00,Coinbase Pro Adds Chainlink (LINK) Cryptocurrency,https://btcmanager.com/coinbase-pro-chainlink-...


# CryptoControl API

In [15]:
# data pulling function

def jsoncleaning(news):
    cryptocontrolnews = pd.DataFrame(news)
    cryptocontrolnews["time"] = pd.to_datetime(cryptocontrolnews["publishedAt"],unit="ns").dt.round("T")
    cryptocontrolnews = cryptocontrolnews.sort_index(by = "time",ascending=True)
    cryptocontrolnews = cryptocontrolnews.assign(noofnews=lambda x: len(x["title"])/len(x["title"]))
    #cryptocontrolnews["time_15"] = cryptocontrolnews["time"].dt.round("15min")
    cryptocontrolnews["time_60"] = cryptocontrolnews["time"].dt.round("60min")
    cryptocontrolnews["number"] = cryptocontrolnews.groupby(["time_60"])["noofnews"].count()
    #cryptocontrolnews["number"] = cryptocontrolnews["noofnews"].groupby(pd.TimeGrouper(freq=("30Min"))).count()
    cryptocontrolnews["count"] = cryptocontrolnews["noofnews"].cumsum()
    cryptocontrolnews = cryptocontrolnews.fillna(0)
    cryptocontrolnews.set_index("time", inplace=True)
    
    
    
    return cryptocontrolnews

In [16]:
# https://github.com/cryptocontrol/python-api

from crypto_news_api import CryptoControlAPI
api = CryptoControlAPI("249912434401b70b8cd41dd8d6e0a876")
btc_new = api.getTopNewsByCoin("bitcoin")
news = api.getTopNews()

In [17]:
cryptocontrolnews = jsoncleaning(news)
cryptocontrolnews_number = cryptocontrolnews.groupby(["time_60"])["noofnews"].count()

In [18]:
CryptoControlAPI = cryptocontrolnews[['description', 'hotness','title',"noofnews","count","time_60"]].head()
CryptoControlAPI.reset_index(inplace = True)

In [19]:
CryptoControlAPI = CryptoControlAPI[['time','description','title']]
#CryptoControlAPI = CryptoControlAPI[['time','description','title']]


In [20]:
CryptoControlAPI.columns=['Time','Text','Title']

In [21]:
CryptoControlAPI

,Time,Text,Title
0,2019-06-27 06:07:00,At the opening of the market today on the dail...,"GBPUSD Is Consolidating at $1.26 Level, Expect..."
1,2019-06-27 07:58:00,The thirteenth stock exchange in the world fea...,Swiss exchange SIX to use R3 for ERC20 token t...
2,2019-06-27 10:01:00,Hackers have hijacked the official Twitter han...,"Tesco’s Twitter Account Compromised, Hacker As..."
3,2019-06-27 10:43:00,2018 was a major bearish year in the crypto ma...,Hackers Target Singapore’s Bitrue stealing $4....
4,2019-06-27 11:41:00,Coinbase Pro is actively expanding the range o...,Chainlink (LINK) is Now Trading on Coinbase...


# Crypto compare

In [22]:
# Function with API to pull news data from Coinstats

def pullnews_cryptocompare(url):
    response = requests.get(url)
    data = response.json()["Data"]
    cryptocompare_News = pd.DataFrame(data)
    
    # Parse data from API
    ts = cryptocompare_News["published_on"]
    
    date = []
    for i in ts:
        b = int(i)
        date.append(datetime.fromtimestamp(b).strftime('%Y-%m-%d %H:%M:%S'))
    
    date = pd.DataFrame(date, columns = ["time"])
    
    cryptocompare_news = pd.concat([date, cryptocompare_News], axis=1)
    cryptocompare_news["time"]  = cryptocompare_news["time"].astype("datetime64[ns]").dt.round("T")
    cryptocompare_news = cryptocompare_news.sort_index(ascending=False)
    cryptocompare_news = cryptocompare_news.assign(noofnews=lambda x: len(x["title"])/len(x["title"]))
    cryptocompare_news["time_60"] = cryptocompare_news["time"].dt.round("60min")
    cryptocompare_news["number"] = cryptocompare_news.groupby(["time_60"])["noofnews"].count()
    cryptocompare_news["count"] = cryptocompare_news["noofnews"].cumsum()
    cryptocompare_news = cryptocompare_news.fillna(0)
    cryptocompare_news.set_index("time", inplace=True)
    
    return cryptocompare_news

In [23]:
url = "https://min-api.cryptocompare.com/data/v2/news/?lang=EN"
cryptocompare_news = pullnews_cryptocompare(url)

In [24]:
Cryptocompare = cryptocompare_news[["id","source","title","upvotes","noofnews"]]
Cryptocompare.reset_index(inplace = True)

In [25]:
#Cryptocompare = Cryptocompare[(Cryptocompare['source'] != 'coindesk') &(Cryptocompare['source'] != 'cointelegraph')  ]

Cryptocompare=Cryptocompare[['time','title']]
Cryptocompare.columns = ['Time','Title']

In [26]:
Cryptocompare.head()

,Time,Title
0,2019-06-28 07:55:00,Gemini Opens Chicago Offices 2 Months After Co...
1,2019-06-28 07:56:00,Litecoin Price Analysis: Litecoin (LTC) Loses ...
2,2019-06-28 08:01:00,Bitcoin Rally Unravels: Here Is Why $10K Holds...
3,2019-06-28 08:08:00,Bitcoin Price Analysis: Bitcoin (BTC) Starts D...
4,2019-06-28 08:19:00,Monero Price Analysis: Bit of Stability In Fal...


# Coin Gecko

In [27]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

news = cg.get_status_updates()

In [28]:
def jsoncleaning(news):
    coingeckonews = pd.DataFrame(news)
    coingeckonews["time"] = pd.to_datetime(coingeckonews["created_at"],unit="ns").dt.round("T")
    coingeckonews = coingeckonews.sort_index(by = "time",ascending=True)
    coingeckonews = coingeckonews.assign(noofnews=lambda x: len(x["description"])/len(x["description"]))
    coingeckonews["time_60"] = coingeckonews["time"].dt.round("60min")
    coingeckonews["number"] = coingeckonews.groupby(["time_60"])["noofnews"].count()
    #cryptocontrolnews["number"] = cryptocontrolnews["noofnews"].groupby(pd.TimeGrouper(freq=("30Min"))).count()
    coingeckonews["count"] = coingeckonews["noofnews"].cumsum()
    coingeckonews = coingeckonews.fillna(0)
    coingeckonews.set_index("time", inplace=True)
    
    
    
    return coingeckonews

In [29]:
coingeckonews = jsoncleaning(news['status_updates'])

In [30]:
CoinGecko = coingeckonews[["description","pin","user_title","noofnews"]]
CoinGecko.reset_index(inplace = True)
CoinGecko = CoinGecko[['time','description']]
CoinGecko.columns=['Time','Text']

In [31]:
CoinGecko.head()

,Time,Text
0,2019-06-25 12:07:00,Here's our founder Kain's response to today's ...
1,2019-06-25 15:08:00,We made some improvements to our website:\r\n\...
2,2019-06-25 15:19:00,"DigiByte will be listed on ZBG, (World’s top 1..."
3,2019-06-25 15:30:00,"BitoPro Trading Competition: Win 33,333 USDT T..."
4,2019-06-25 15:38:00,DigiByte is live on NOWPayments.\r\nNowPayment...


# Messari

In [32]:
def pullnews_messari(url):
    
    response = requests.get(url)
    data = response.json()
    data1 = data["data"]
    messari_news = pd.DataFrame(data1)
    
    # Parse data from API
    messari_news["time"] = pd.to_datetime(messari_news["published_at"]).dt.round("T")
    messari_news = messari_news.sort_index(ascending=False)
    messari_news = messari_news.assign(noofnews=lambda x: len(x["title"])/len(x["title"]))
    #messari_news["time_60"] = messari_news["feedDate"].dt.round("60min")
    messari_news["number"] = messari_news.groupby(["published_at"])["noofnews"].count()
    messari_news["count"] = messari_news["noofnews"].cumsum()
    messari_news = messari_news.fillna(0)
    messari_news.set_index("time", inplace=True)
    
    return messari_news

In [33]:
url = "https://data.messari.io/api/v1/news"

messari_news = pullnews_messari(url)
messari_news = messari_news[['title','url','content']].rename(columns={'title':'Title','url':'Link','content':'Text'})
messari_news.reset_index(inplace = True)

In [34]:
messari_news.columns=['Time','Title','Links','Text']

In [35]:
messari_news

,Time,Title,Links,Text
0,2019-06-24,Microsoft and Ethereum Foundation join Hyperle...,https://messari.io/article/microsoft-and-ether...,Microsoft and the Ethereum Foundation are some...
1,2019-06-24,[Analysis] A Monumental Fight Over Facebook's ...,https://messari.io/article/analysis-a-monument...,The reaction from lawmakers to Facebook's Libr...
2,2019-06-24,"Florida city agrees to pay $600,000 in bitcoin...",https://messari.io/article/florida-city-agrees...,The New York Times reports that a Florida city...
3,2019-06-24,Iranian official warns that cryptocurrency min...,https://messari.io/article/iranian-official-wa...,An official from Iran's state-run power genera...
4,2019-06-24,Zcash executives fire back at Facebook’s attac...,https://messari.io/article/zcash-executives-fi...,"Last week, Facebook’s blockchain lead David Ma..."
5,2019-06-24,[Messari Research] Coinbase-Neutrino-Chainaly...,https://messari.io/article/messari-research-co...,*This was originally sent to subscribers of Me...
6,2019-06-24,Trump economist thinks Facebook’s Libra is ‘a ...,https://messari.io/article/trump-economist-thi...,"Facebook’s upcoming Libra cryptocurrency is ""a..."
7,2019-06-24,Malta's Prime Minister announces every rental ...,https://messari.io/article/malta-s-prime-minis...,"Joseph Muscat, the Prime Minister of Malta, an..."
8,2019-06-24,[Community Post] Long Reads Sunday - Issue #52,https://messari.io/article/community-post-long...,Community posts are hand selected contribution...
9,2019-06-24,[Analysis] An Analysis of Kin’s On-Chain Activ...,https://messari.io/article/analysis-an-analysi...,Coin Metrics analyzed Kin's blockchain in orde...


# News api

In [36]:
from newsapi.newsapi_client import NewsApiClient
api = NewsApiClient(api_key="71008a0ff7c644c5b38d071ebd502011")
newsapi = api.get_top_headlines(q = "bitcoin")

In [37]:
newsapi

{'status': 'ok',
 'totalResults': 3,
 'articles': [{'source': {'id': 'the-next-web', 'name': 'The Next Web'},
   'author': 'Yessi Bello Perez',
   'title': 'Bitcoin dark web drug ring busted after dealer doxes himself buying stamps',
   'description': 'US law enforcers\xa0have busted a Bitcon-fuelled dark web drug ring after one of the alleged dealers messed up and bought pre-printed postage online using his real\xa0identity. The dealers, which sold almost $9 million worth of counterfeit Adderall, leveraged the a…',
   'url': 'https://thenextweb.com/hardfork/2019/06/28/bitcoin-dark-web-drug-ring-busted-after-dealer-doxes-himself-buying-stamps/',
   'urlToImage': 'https://img-cdn.tnwcdn.com/image/hardfork?filter_last=1&fit=1280%2C640&url=https%3A%2F%2Fcdn0.tnwcdn.com%2Fwp-content%2Fblogs.dir%2F1%2Ffiles%2F2019%2F06%2FUntitled-design-49.jpg&signature=cfd713f90fb3fab447dd2978a8a14ce5',
   'publishedAt': '2019-06-28T08:44:15Z',
   'content': 'US law enforcers\xa0have busted a Bitcon-fuelle

In [38]:
def pullnews_newsapiclient(newsapi):
    newsapiclient = pd.DataFrame(newsapi)
    # Parse data from API
    newsapiclient["time"] = pd.to_datetime(newsapiclient["publishedAt"],unit="ns").dt.round("T")
    newsapiclient = newsapiclient.sort_index(ascending=False)
    newsapiclient = newsapiclient.assign(noofnews=lambda x: len(x["title"])/len(x["title"]))
    newsapiclient["time_60"] = newsapiclient["time"].dt.round("60min")
    newsapiclient["number"] = newsapiclient.groupby(["time_60"])["noofnews"].count()
    newsapiclient["count"] = newsapiclient["noofnews"].cumsum()
    newsapiclient = newsapiclient.fillna(0)
    newsapiclient.set_index("time", inplace=True)
    #newsapiclient = newsapiclient.sort_index(ascending=True)
    
    return newsapiclient

In [39]:
newsapiclient = pullnews_newsapiclient(newsapi["articles"])
newsapiclient.reset_index(inplace = True)
#newsapiclient=newsapiclient[['time','content','title','url']]
newsapiclient

,time,author,content,description,publishedAt,source,title,url,urlToImage,noofnews,time_60,number,count
0,2019-06-28 04:41:00,Joseph Young,"In the past 24 hours, the valuation of the cry...","In the past 24 hours, the valuation of the cry...",2019-06-28T04:40:57Z,"{'id': 'crypto-coins-news', 'name': 'Crypto Co...",Spiralling Bitcoin Pullback Snowballs to Bruta...,https://www.ccn.com/crypto/bitcoin-pullback-br...,https://www.ccn.com/wp-content/uploads/2019/06...,1.0,2019-06-28 05:00:00,0.0,1.0
1,2019-06-28 08:42:00,Matthew Beedham,"Authorities in Iran have seized around 1,000 B...","Authorities in Iran have seized around 1,000 B...",2019-06-28T08:41:49Z,"{'id': 'the-next-web', 'name': 'The Next Web'}","Iran seizes 1,000 Bitcoin miners, says mining ...",https://thenextweb.com/hardfork/2019/06/28/ira...,https://img-cdn.tnwcdn.com/image/hardfork?filt...,1.0,2019-06-28 09:00:00,0.0,2.0
2,2019-06-28 08:44:00,Yessi Bello Perez,US law enforcers have busted a Bitcon-fuelled ...,US law enforcers have busted a Bitcon-fuelled ...,2019-06-28T08:44:15Z,"{'id': 'the-next-web', 'name': 'The Next Web'}",Bitcoin dark web drug ring busted after dealer...,https://thenextweb.com/hardfork/2019/06/28/bit...,https://img-cdn.tnwcdn.com/image/hardfork?filt...,1.0,2019-06-28 09:00:00,0.0,3.0


In [40]:
newsapiclient=newsapiclient[['time','content','title','url']]
newsapiclient.columns = ['Time','Text','Title','Links']

In [41]:
newsapiclient


,Time,Text,Title,Links
0,2019-06-28 04:41:00,"In the past 24 hours, the valuation of the cry...",Spiralling Bitcoin Pullback Snowballs to Bruta...,https://www.ccn.com/crypto/bitcoin-pullback-br...
1,2019-06-28 08:42:00,"Authorities in Iran have seized around 1,000 B...","Iran seizes 1,000 Bitcoin miners, says mining ...",https://thenextweb.com/hardfork/2019/06/28/ira...
2,2019-06-28 08:44:00,US law enforcers have busted a Bitcon-fuelled ...,Bitcoin dark web drug ring busted after dealer...,https://thenextweb.com/hardfork/2019/06/28/bit...


In [42]:
#newsapiclient.resample("60T")["noofnews"].count().iplot(kind="line")

# Scoring of news

In [43]:
# https://github.com/HarsimratBhundar/Cryptocurrency-Price-Prediction-Sentiment-Analysis/blob/master/CryptoPrediction/news_senti.py

from newsapi.newsapi_client import NewsApiClient
import json
import os
import pandas as pd
import requests
from datetime import timedelta, date
from textblob import TextBlob
from dateutil import parser

# replace your api key with '#############' in the following line of code
newsapi = NewsApiClient(api_key='71008a0ff7c644c5b38d071ebd502011')

def get_articles(topic, channels, date):
    return newsapi.get_everything(q=topic
                            )

def get_sent_analysis(topic, channels, from_date, to_date):
    results = pd.DataFrame()
    
    for date in pd.date_range(from_date, to_date):
        #print(date.strftime('%Y.%m.%d'))
        all_articles = get_articles(topic, channels, date.strftime('%Y-%m-%d'))
        results = results.append(pd.DataFrame(all_articles['articles']))
        data = results

    results['polarity'] = results.apply(lambda x: TextBlob(str(x['description'])).sentiment.polarity, axis = 1)
    results['subjectivity'] = results.apply(lambda x: TextBlob(str(x['description'])).sentiment.subjectivity, axis = 1)
    results['date'] = results.apply(lambda x: parser.parse(x['publishedAt']).strftime('%Y.%m.%d'), axis = 1)
    results['time'] = results.apply(lambda x: parser.parse(x['publishedAt']).strftime('%H:%M'), axis = 1)
    
    return results.groupby(by = 'date').mean(), data 

#df = get_articles['articles']
df1 = get_sent_analysis('bitcoin', 'bbc-news, the verge, financial-times, reuters, metro, cointelegraph, crypto-coins-news, bloomberg, cnbc, business-insider, google-news, forbes, fortune', date(2019,5,15), date(2019,6,6))





In [44]:
from newsapi.newsapi_client import NewsApiClient
import json
import os
import pandas as pd
import requests
from datetime import timedelta, date
from textblob import TextBlob
from dateutil import parser

# replace your api key with '#############' in the following line of code
newsapi = NewsApiClient(api_key='71008a0ff7c644c5b38d071ebd502011')

def get_articles(topic, channels, date):
    return newsapi.get_everything(q=topic)

def get_sent_analysis(topic, channels, from_date, to_date):
    results = pd.DataFrame()
    
    for date in pd.date_range(from_date, to_date):
        #print(date.strftime('%Y.%m.%d'))
        all_articles = get_articles(topic, channels, date.strftime('%Y-%m-%d'))
        results = results.append(pd.DataFrame(all_articles['articles']))
        data = results

    results['polarity'] = results.apply(lambda x: TextBlob(str(x['description'])).sentiment.polarity, axis = 1)
    results['subjectivity'] = results.apply(lambda x: TextBlob(str(x['description'])).sentiment.subjectivity, axis = 1)
    results['date'] = results.apply(lambda x: parser.parse(x['publishedAt']).strftime('%Y.%m.%d'), axis = 1)
    results['time'] = results.apply(lambda x: parser.parse(x['publishedAt']).strftime('%H:%M'), axis = 1)
    
    return results.groupby(by = 'date').mean(), results['description'], results['date'],results['time']  

#df = get_articles['articles']
df1 = get_sent_analysis('bitcoin', 'bbc-news, the verge, financial-times, reuters, metro, cointelegraph, crypto-coins-news, bloomberg, cnbc, business-insider, google-news, forbes, fortune', date(2019,5,15), date(2019,6,6))


In [45]:
df1

(            polarity  subjectivity
 date                              
 2019.05.27  0.312500      0.737500
 2019.05.28  0.133333      0.745833
 2019.05.29  0.500000      0.500000
 2019.06.03 -0.086667      0.493333
 2019.06.06  0.133333      0.155556
 2019.06.11  0.117857      0.325000
 2019.06.12  0.125000      0.125000
 2019.06.13  0.000000      0.000000
 2019.06.14 -0.080000      0.356667
 2019.06.19  0.003030      0.185795
 2019.06.20  0.000000      0.000000
 2019.06.21  0.252904      0.634217
 2019.06.22 -0.134667      0.234667
 2019.06.26  0.013333      0.303333
 2019.06.27  0.157407      0.375926,
 0     Riviera Beach, a city in Florida, is set to pa...
 1     The cryptocurrency has grown quadruply within ...
 2     Riviera Beach, Fla., authorized its insurer to...
 3     On Tuesday Facebook announced Libra, a cryptoc...
 4     Early last month, hackers infected somewhere a...
 5     Blockchain is the technology behind cryptocurr...
 6     When bitcoin was born it was a symbol 

# BBC

In [46]:
def get_links(search_term):
    df = pd.DataFrame()
    html_page = urllib.request.urlopen("https://www.bbc.co.uk/search?q=" + search_term)
    soup = BeautifulSoup(html_page)
    x = soup.find_all(class_='search-results results')
    Link =[]
    Title =[]
    Time =[]
    Article =[]
    y = x[0].find_all('li')
    for i in y:
        link = i.find('h1').find('a').get('href')
        Link.append(link)
        title = i.find('h1').find('a').getText()
        Title.append(title)
        time = i.find('time').get('datetime')
        Time.append(pd.to_datetime(time))

    Article =[]

    for i in Link:
        res = requests.get(i)
        soup = bs(res.content,'lxml')
        foo = soup.find_all(class_='story-body__inner')
        if len(foo)>0:
            p = foo[0].find_all('p')
            Text =[]
            for i in p:
                text = re.sub(r'&.*?;','', i.getText())
                Text.append(text)
            Text = ''.join(Text)
            
        else:
            foo = soup.find_all(class_='map__intro__synopsis centi')
            if len(foo)>0:
                p = foo[0].find_all('p')
                Text =[]
                for i in p:
                    
                    text = i.getText()
                    Text.append(text)
                Text = ''.join(Text)    
            else:
                Text = 'None'
        Article.append(Text)
    
    df['Title'] = Title
    df['Link'] = Link
    df['Time'] = Time
    df['Text'] = Article
    return df

In [47]:
bbc = get_links('bitcoin')
bbc.columns=['Title','Links','Time','Text']

In [48]:
bbc.head()

,Title,Links,Time,Text
0,Second US town pays up to ransomware hackers,https://www.bbc.co.uk/news/technology-48770128,2019-06-26 10:01:32+00:00,"A town in Florida has paid $500,000 (£394,000)..."
1,50 Things That Made the Modern Economy: Blockc...,https://www.bbc.co.uk/programmes/w3csz2wh,2019-06-29 00:00:00+00:00,Billions are being poured into startups workin...
2,Business Daily: Life in an unrecognised state,https://www.bbc.co.uk/programmes/w3csy738,2019-06-24 00:00:00+00:00,How do you do business with the rest of the wo...
3,Tech Tent: Is Facebook the future of money?,https://www.bbc.co.uk/news/technology-48725512,2019-06-21 16:50:29+00:00,Has Facebook's plan for a new crypto-currency ...
4,Tech Tent: Libra: Facebook's digital coin,https://www.bbc.co.uk/programmes/w3csymsf,2019-06-21 00:00:00+00:00,How payments and money transfers could be a bi...


In [49]:
df = LiveCoinWatch.append([Coindesk,Forexlive,Coinlore,Cointelegraph,Coinstat,CryptoControlAPI,Cryptocompare,CoinGecko,messari_news,bbc,newsapiclient],ignore_index = True)

In [50]:
df.shape

(312, 5)

In [51]:
df = df.drop_duplicates(subset = 'Title',keep = 'first')

In [52]:
df

,Comment,Links,Text,Time,Title
0,0,https://news.livecoinwatch.com/week-crypto-btc...,No Text,2019-06-22 05:11:57+00:00,"This Week In Crypto: BTC Tops $10,000, Ripple ..."
1,0,https://news.livecoinwatch.com/new-cryptocurre...,One of the leading cryptoc...,2019-06-17 23:00:14+00:00,New cryptocurrencies on 1xBit: stablecoins com...
2,0,https://news.livecoinwatch.com/this-week-in-cr...,This Week In Crypto is a w...,2019-06-15 18:00:50+00:00,This Week In Crypto: Google Lauds Ethereum-bas...
3,0,https://news.livecoinwatch.com/this-week-crypt...,This Week In Crypto is a w...,2019-06-08 04:38:00+00:00,"This Week In Crypto: India Bitcoin Ban, EOS Vo..."
4,0,https://news.livecoinwatch.com/week-crypto-bit...,This Week In Crypto is a w...,2019-06-01 01:50:08+00:00,"This Week In Crypto: Bitcoin Bulls Roar, Priva..."
5,NaN,https://www.coindesk.com/trump-administration-...,No Text,2019-06-27 23:31:14+02:00,Trump Administration in Talks With Crypto Star...
6,NaN,https://www.coindesk.com/a-wannabe-netflix-rai...,No Text,2019-06-28 10:00:19+02:00,A Wannabe Netflix Raised $575 Million on Ether...
7,NaN,https://www.coindesk.com/us-lawmakers-question...,No Text,2019-06-27 23:47:47+02:00,US Lawmakers Question Terrorist Use of Faceboo...
8,NaN,https://www.coindesk.com/down-1-7k-bitcoin-div...,No Text,2019-06-28 03:30:42+02:00,Down $1.7K: Bitcoin’s Price Dives Amid Crypto ...
9,NaN,https://www.coindesk.com/irish-crypto-exchange...,Irish cryptocurrency exchange Bitsane may have...,2019-06-28 11:00:01+02:00,Irish Crypto Exchange Bitsane May Have Disappe...
